score = 100
        - w1*road_class_risk
        - w2*speed_risk
        + w3*lighting_bonus
        + w4*sidewalk_bonus
        + w5*signals_crossings_bonus
        

In [3]:
import json
import pandas as pd

# Load your raw OSM json
with open("data/raw/osm_bengaluru_20250815.json", "r", encoding="utf-8") as f:
    osm_data = json.load(f)

# Convert OSM elements into a dataframe of ways
elements = [el for el in osm_data["elements"] if el["type"] == "way"]

rows = []
for el in elements:
    tags = el.get("tags", {})
    rows.append({
        "id": el["id"],
        "highway": tags.get("highway"),
        "maxspeed": tags.get("maxspeed"),
        "lit": tags.get("lit"),
        "sidewalk": tags.get("sidewalk"),
        "n_nodes": len(el.get("nodes", []))
    })

df = pd.DataFrame(rows)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/osm_bengaluru_20250815.json'